In [1]:
import pandas as pd
import datetime

# work_time_table = pd.read_csv('./근무시간_테이블.csv')

In [3]:
# 날짜로 요일 계산하기
days_dic = {
    1: '월요일',
    2: '화요일',
    3: '수요일',
    4: '목요일',
    5: '금요일',
    6: '토요일',
    7: '일요일'
}

In [10]:
now = datetime.datetime.now()

today = datetime.datetime.strftime(now, '%Y%m%d')
today_weekday = days_dic[now.isoweekday()]

In [24]:
now.isocalendar().week

18

In [23]:
now.isoweekday()

2

In [ ]:
input_day = None

In [35]:
work_time_table

,date,weekday,start_time,end_time,spend_time,rest_time


In [55]:
work_time_table.loc[0] = ['20240430', np.nan, np.nan, np.nan, np.nan, np.nan]

In [56]:
work_time_table

,date,weekday,start_time,end_time,spend_time,rest_time
0,20240430,NaN,NaN,NaN,NaN,NaN


In [68]:
df_ = pd.DataFrame([['20240430', np.nan, np.nan, np.nan, np.nan, np.nan]],
                   columns = ['date', 'weekday', 'start_time', 'end_time', 'spend_time', 'rest_time'])

In [69]:
pd.concat([work_time_table, df_])

,date,weekday,start_time,end_time,spend_time,rest_time
0,20240430,NaN,NaN,NaN,NaN,NaN
0,20240430,NaN,NaN,NaN,NaN,NaN


In [237]:
class working_time_calculator:
    def __init__(self):
#         self.work_time_table = pd.read_csv(file_path)
        self.work_time_table = pd.DataFrame(columns = ['date', 'month', 'week_num', 'weekday', 'start_time', 'end_time', 'spend_time', 'rest_time'])
        self.rest_time = 1
        
        self.days_dic = {
                            1: '월요일',
                            2: '화요일',
                            3: '수요일',
                            4: '목요일',
                            5: '금요일',
                            6: '토요일',
                            7: '일요일'
        }
        
        return
    
    # 현재 시점 기준으로 업무 시작, 종료를 입력하는 함수
    def work_start(self, start_time = None):
        input_time = datetime.datetime.now()

        input_weekday = self.days_dic[input_time.isoweekday()]
        input_weeknum = input_time.isocalendar().week
        input_day = datetime.datetime.strftime(input_time, '%Y%m%d')
        input_time = datetime.datetime.strftime(input_time, '%Y-%m-%d %H:%M')
        
        self.work_time_table.loc[len(self.work_time_table)] =\
        [input_day, input_weeknum, input_weekday, input_time, np.nan, np.nan, self.rest_time]
        
        return
        
    def work_end(self, end_time = None):
        input_time = datetime.datetime.now()
        
        input_day = datetime.datetime.strftime(input_time, '%Y%m%d')
        input_time = datetime.datetime.strftime(input_time, '%Y-%m-%d %H:%M')
        
        # spend time 게산
        start_time = self.work_time_table.loc[self.work_time_table['date'] == input_day]['start_time'].values[0]
        
        start_time = datetime.datetime.fromisoformat(start_time)
        end_time = datetime.datetime.fromisoformat(input_time)
        
        spend_time = end_time - start_time - datetime.timedelta(hours = 1)
        spend_time = spend_time.total_seconds()
        
        # 한시간을 안넘을 경우, - 값이 나와서 값이 정확하게 안나옴
        spend_hours = int(spend_time // 3600)
        spend_minutes = int((spend_time % 3600) // 60)
        
        spend_time = f'{spend_hours:02d}:{spend_minutes:02d}'
        
        # input
        self.work_time_table.loc[self.work_time_table['date'] == input_day, ('end_time', 'spend_time')] = (input_time, spend_time)
        
        return
    
    
    def insert_work_time(self, start_time, end_time = '', rest_time = 1, input_day = None): # time은 HH:MM 형태의 스트링으로 입력
        # 날짜 계산
        if input_day == None:
            input_day = datetime.datetime.now()

            input_weekday = days_dic[input_day.isoweekday()]
            input_weeknum = input_day.isocalendar().week
            input_day = datetime.datetime.strftime(input_day, '%Y%m%d')

        else:
            input_weekday = days_dic[datetime.datetime.strptime(input_day, '%Y%m%d').isoweekday()]
            input_weeknum = input_day.isocalendar().week

        # 시간 계산
        if end_time == '': # 퇴근 시간을 입력하지 않을 경우 현재 시점으로 계산
            end_time = datetime.datetime.now()
#             end_time =

In [238]:
a = working_time_calculator()

In [239]:
a.work_start()

In [243]:
a.work_end()

In [244]:
a.work_time_table

,date,week_num,weekday,start_time,end_time,spend_time,rest_time
0,20240430,18,화요일,2024-04-30 16:46,2024-04-30 16:47,-1:01,1


In [135]:
def work_time(start_time, finish_time, break_time): # HH:MM 형태의 스트링으로 입력
    if finish_time == '':
        finish_time = datetime.datetime.now()
        finish_time = f'{finish_time.hour:02d}:{finish_time.minute}'
                
    finish_time = datetime.timedelta(
                                        hours = int(finish_time.split(':')[0]),
                                        minutes = int(finish_time.split(':')[1])
    )
    
    start_time = datetime.timedelta(
                                        hours = int(start_time.split(':')[0]),
                                        minutes = int(start_time.split(':')[1])
    )
    
    break_time = datetime.timedelta(hours = break_time)

    spend_time = finish_time - start_time - break_time
    
    return spend_time

In [132]:
# day = 
day =  '수요일'

In [138]:
# start_time = input('출근 시간을 입력하세요(HH:MM).')
# finish_time = input('퇴근 시간을 입력하세요(HH:MM).')
# break_time = 1

spend_time = work_time(start_time, finish_time, break_time)

# 포맷화
# spend_time = f"{int(str(spend_time).split(':')[0]):02d}:{str(spend_time).split(':')[1]}"

In [142]:
spend_time

datetime.timedelta(seconds=18960)

In [ ]:
work_

In [133]:
now = datetime.datetime.now()

today = now.strftime('%Y%m%d')
if day == None:
    day = now.weekday()
    day = days_dic[day]
    
work_time_table = work_time_table.loc[work_time_table['요일'] != day]

start_time = input('출근 시간을 입력하세요(HH:MM).')
finish_time = input('퇴근 시간을 입력하세요(HH:MM).')
break_time = 1

spend_time = work_time(start_time, finish_time, break_time)

# 포맷화
spend_time = f"{int(str(spend_time).split(':')[0]):02d}:{str(spend_time).split(':')[1]}"

data = [today, day, start_time, finish_time, str(spend_time), break_time]

work_time_table.loc[len(work_time_table)] = data

KeyError: '요일'

In [10]:
display(work_time_table)

spend_sum = work_time_table['근무시간'].apply(lambda x: datetime.timedelta(
                                                            hours = int(x.split(':')[0]),
                                                            minutes = int(x.split(':')[1])
                           )).sum()

left_time = work_time_week - spend_sum
left_time_h = left_time.total_seconds() // 3600
left_time_m = (left_time.total_seconds() % 3600) // 60

print(f'남은 근무 시간은 {int(left_time_h):02d}:{int(left_time_m):02d} 입니다.')

,일자,요일,출근시간,퇴근시간,근무시간,휴게시간
0,20240418,월요일,09:43,20:19,09:36,1
1,20240418,화요일,09:44,18:02,07:18,1
2,20240418,수요일,09:43,20:08,09:25,1


남은 근무 시간은 13:41 입니다.


In [2]:
start_time = '09:42'
time_plan = '6:13'

start_hour = int(start_time.split(':')[0])
start_minute = int(start_time.split(':')[1])
start_time = datetime.timedelta(
                                hours = start_hour,
                                minutes = start_minute
)

plan_hour = int(time_plan.split(':')[0])
plan_minute = int(time_plan.split(':')[1])
plan_time = datetime.timedelta(
                                hours = plan_hour,
                                minutes = plan_minute
)

break_time = datetime.timedelta(hours = 1)

finish_time = start_time + plan_time + break_time
finish_time_h = finish_time.total_seconds() // 3600
finish_time_m = (finish_time.total_seconds() % 3600) // 60
print(f"오늘의 퇴근 예정 시간은 {int(finish_time_h):02d}:{int(finish_time_m)} 입니다.")

오늘의 퇴근 예정 시간은 16:55 입니다.


In [16]:
(datetime.timedelta(hours = 39, minutes = 33) + datetime.timedelta(hours = 32, minutes = 41)).total_seconds()

260040.0

In [17]:
260040.0 // 3600

72.0

In [22]:
(260040.0 % 3600) // 60

14.0